In [1]:
import os
os.chdir('..')

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig, GenerationConfig
from src.alignment.model_utils import get_tokenizer
from src.alignment.configs import ModelArguments, DataArguments

/home/ubuntu/miniconda3/envs/lang/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ubuntu/miniconda3/envs/lang/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/ubuntu/miniconda3/envs/lang/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/ubuntu/miniconda3/envs/lang/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytr

In [3]:
# config = AutoConfig.from_pretrained('/home/ubuntu/hieu.nn/alignment-handbook/outputs/models/pythia-410m-sft-full/final-model')

In [46]:
model_args = ModelArguments(
    model_name_or_path='/home/ubuntu/hieu.nn/alignment-handbook/outputs/models/pythia-410m-sft-full/final-model',
    # model_name_or_path='EleutherAI/pythia-410m',
    torch_dtype="bfloat16",
    # torch_dtype="float16",
)

torch_dtype = (
    model_args.torch_dtype if model_args.torch_dtype in ["auto", None] else getattr(torch, model_args.torch_dtype)
)

model_kwargs = dict(
    revision=model_args.base_model_revision,
    trust_remote_code=model_args.trust_remote_code,
    use_flash_attention_2=model_args.use_flash_attention_2,
    torch_dtype=torch_dtype,
    # use_cache=False if training_args.gradient_checkpointing else True,
    # device_map=get_kbit_device_map() if quantization_config is not None else None,
    # quantization_config=quantization_config,
)



In [47]:
model = AutoModelForCausalLM.from_pretrained(
            model_args.model_name_or_path,
            **model_kwargs
)
tokenizer = get_tokenizer(model_args, DataArguments())


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [48]:
genconf = GenerationConfig.from_pretrained(model_args.model_name_or_path)
genconf.max_length=200

In [49]:
model = model.to("cuda:5")

In [50]:
sent = torch.tensor(tokenizer.encode("Alice was so tired when she got back home so she went")).unsqueeze_(0).to("cuda:5")

In [51]:
with torch.no_grad():
    # out = model.generate(sent, genconf)
    out = model.generate(sent, max_length=200)

/home/ubuntu/miniconda3/envs/lang/lib/python3.11/site-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [52]:
print(tokenizer.decode(out.tolist()[0]))

Alice was so tired when she got back home so she went to bed early. She was tired of the constant noise and the constant worry. She didn't want to be a burden to her family.

As she lay in bed, she thought about the day she had spent with her parents. She had been so happy and so carefree. She had been so happy to be with her parents. She had been so happy to be with her parents.

But then, she had been so worried. She had been so worried about her parents. She had been so worried about her parents.

She had been so worried about her parents.

She had been so worried about her parents.

She had been so worried about her parents.

She had been so worried about her parents.

She had been so worried about her parents.

She had been so worried about her parents.

She had been so worried about her parents.

